#Librerías

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import etree
from datetime import date
from datetime import datetime
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Fecha de hoy
fecha_hoy = datetime.today().strftime('%Y_%m_%d')
print(fecha_hoy)

# Scrapeo paginas

In [ ]:
%%time
# base_url = "https://www.argenprop.com/departamentos/alquiler/capital-federal-o-villa-urquiza?pagina-" #ACA SE DEFINEN LAS PAGINAS A SCRAPEAR
# base_url = "https://www.argenprop.com/departamentos-o-ph/alquiler/capital-federal?pagina-"  #Acá se define el tipo de propiedades, la operacion y la zona
base_url = "https://www.argenprop.com/casas-o-departamentos-o-ph/capital-federal?pagina-"  #Acá se define el tipo de propiedades, la operacion y la zona


pagina_numero = 1 #Genero un contador de páginas

df = pd.DataFrame({ "Clase":[], "Barrio": [], "Ubicacion":[],"Descripcion":[],"Precio":[], "Links":[]}) #CReo un DF vacío. Si se agregan nuevas columnas, agregar acá tb

while True: #While loop que recorra todas las páginas hasta que no encuentre mas
    url = base_url + str(pagina_numero) #URL de la página + nro de página
    resultado = requests.get(url, verify=False)

    if resultado.status_code == 200: #Condicional que chequea si tiene un resultado efectivo
        soup = BeautifulSoup(resultado.text, 'html.parser') #Parsea el html de esa página
        siguiente = soup.find_all('a', attrs={"aria-label":"Siguiente", "rel":"next"}) #Creo una variable con el contenido de la tarjeta "siguiente", si  no la encuentra, esta vacío

        clase = soup.find_all('p', attrs={"class":"card__title--primary hide-mobile"})
        clase = [i.text.strip() for i in clase]

        barrio_ciudad = soup.find_all('p', attrs={"class":"card__title--primary show-mobile"})
        barrio_ciudad = [i.text.strip() for i in barrio_ciudad]

        ubicacion = soup.find_all('p', attrs={"class":"card__address"})
        ubicacion = [i.text.strip() for i in ubicacion]

        descripcion = soup.find_all('h2', attrs={"class":'card__title'})
        descripcion = [i.text.strip() for i in descripcion]

        precios = soup.find_all('p', attrs={"class":'card__price'})
        precios = [i.text.strip() for i in precios]

        links = soup.find_all('a', attrs={'class':'card'})
        links = [i.get('href') for i in links]
        links = list(map(lambda n: f'https://www.argenprop.com{n}', links))

        df_prov = pd.DataFrame({"Clase":clase, "Barrio":barrio_ciudad, "Ubicacion":ubicacion,"Descripcion":descripcion, "Precio":precios, "Links":links})

        print(f"Página {pagina_numero} --> Ok, casos relevados: {df_prov.shape[0]}")

        df = pd.concat([df,df_prov], axis=0)

        if siguiente:
            pagina_numero += 1
        else:
          print(f"Fin en página {pagina_numero}")
          break
    else:
        print(f"Página no encontrada (Error {resultado.status_code})")
        break

df.shape

Página 1 --> Ok, casos relevados: 20
Página 2 --> Ok, casos relevados: 20
Página 3 --> Ok, casos relevados: 20
Página 4 --> Ok, casos relevados: 20
Página 5 --> Ok, casos relevados: 20
Página 6 --> Ok, casos relevados: 20
Página 7 --> Ok, casos relevados: 20
Página 8 --> Ok, casos relevados: 20
Página 9 --> Ok, casos relevados: 20
Página 10 --> Ok, casos relevados: 20
Página 11 --> Ok, casos relevados: 20
Página 12 --> Ok, casos relevados: 20
Página 13 --> Ok, casos relevados: 20
Página 14 --> Ok, casos relevados: 20
Página 15 --> Ok, casos relevados: 20
Página 16 --> Ok, casos relevados: 20
Página 17 --> Ok, casos relevados: 20
Página 18 --> Ok, casos relevados: 20
Página 19 --> Ok, casos relevados: 20
Página 20 --> Ok, casos relevados: 20
Página 21 --> Ok, casos relevados: 20
Página 22 --> Ok, casos relevados: 20
Página 23 --> Ok, casos relevados: 20
Página 24 --> Ok, casos relevados: 20
Página 25 --> Ok, casos relevados: 20
Página 26 --> Ok, casos relevados: 20
Página 27 --> Ok, cas

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.head()

#Acomodo datos

In [ ]:
#Normalizo precios y ubicacion
df.Precio=df.Precio.str.replace('\n','').str.replace(' ','').str.replace('USD','USD  ')
df.Ubicacion=df.Ubicacion.str.replace('\n','').str.strip(' ').str.strip('\n')

In [ ]:
#Agrego fecha de la consulta
df['fecha_consulta'] = fecha_hoy

In [ ]:
df.sample(10)

In [ ]:
df['Clase'].str.split(' en ',expand=True)

In [ ]:
df.columns.values

In [ ]:
# df = df[['Clase', 'Barrio', 'Ubicacion', 'Descripcion', 'Precio', 'Links',
#        'fecha_consulta']]

In [ ]:
df[['tipo_prop','operacion','Barrio_vf']] = df['Clase'].str.split(' en ', expand=True)
df[['Barrio_vf','localidad']] = df['Barrio_vf'].str.split(',', expand=True)
df

In [ ]:
df.operacion.unique()

In [ ]:
df.operacion.value_counts(normalize=True)

In [ ]:
df.tipo_prop.value_counts(normalize=True)

In [ ]:
df.head()

#Guardar base

In [ ]:
# df.to_csv(f'Argenprop_CABA_{fecha_hoy}.csv')